In [3]:
from deeprpi.utils import RPIDataset

In [10]:
RPIDataset = RPIDataset(data_path='./data/NPInter5.csv',
                            batch_size=32,
                            num_workers=4,
                            rna_col='RNA_aa_code',
                            protein_col='target_aa_code',
                            label_col='Y',
                            padding=True,
                            rna_max_length=1000,
                            protein_max_length=1000,
                            truncation=False,
                            val_ratio=0.1,
                            test_ratio=0.1)

In [11]:
RPIDataset.setup()
train_dataloader = RPIDataset.train_dataloader()

Selecting data: 100%|██████████| 1182/1182 [00:00<?, ?it/s]


Selected 313 samples from 1182 samples


Padding protein sequences: 100%|██████████| 313/313 [00:00<?, ?it/s]
